In [1]:
..\SpyderProject\detector_streaming\venv\Scripts\activate

SyntaxError: invalid syntax (<ipython-input-1-310f1a7c24a4>, line 1)

#https://youtu.be/ywaSiIf1C2w?t=505

#face_detector.py

In [ ]:
import cv2;

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW);
face_detector = cv2.CascadeClassifier(cv2.data.haarcascades +
    "haarcascade_frontalface_default.xml");
while True:
    ret, frame = cap.read();
    if ret:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY);
        faces = face_detector.detectMultiScale(gray, 1.3, 5);
        for(x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2);
        cv2.imshow("PenguinL", frame);
        
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break;

cap.release();
cv2.destroyAllWindows();

#videostreaming_web.py

In [ ]:
from flask import Flask;
from flask import render_template;
from flask import Response;
import cv2;

app = Flask(__name__);
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW);
face_detector = cv2.CascadeClassifier(cv2.data.haarcascades +
    "haarcascade_frontalface_default.xml");

def generate():
    while True:
        ret, frame = cap.read();
        if ret:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY);
            faces = face_detector.detectMultiScale(gray, 1.3, 5);
            for(x, y, w, h) in faces:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2);
            (flag, encodedImage) = cv2.imencode(".jpg", frame);
            if not flag:
                continue
            yield(b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' +
                 bytearray(encodedImage) + b'\r\n');

@app.route("/")
def index():
    return render_template("index.html");

@app.route("/video_feed")
def video_feed():
    return Response(generate(),
        mimetype = "multipart/x-mixed-replace; boundary=frame");


if __name__ == "__main__":
    app.run(host='0.0.0.0', port=1111, debug=False);

cap.release();

#templates/index.html

In [ ]:
<!DOCTYPE html>
<html lang="en">
    <head>
        <meta charset="UTF-8">
<!--         <meta http-equiv="X_UA_Compatible" content="IE-edge"> -->
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>
            PenguinL
        </title>
        <style>
            .container{
                margin: 0;
                padding: 0;
                width: 100%;
                height: 100vh;
                background-color: #E2D8F8;
                color: black;
                text-align: center;
            }
        </style>
    </head>
    <body class = "container">
        <h1>
            Video en directo
        </h1>
        <img src="{{ url_for('video_feed') }}">
    </body>
</html>